In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# plotly visualization library
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly_express as px

# dimensionality decomposition libarires
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP

## Reading files and preprocessing

In [90]:
input_path = '../data/MovementAAL/dataset/MovementAAL_RSS_'
target_path = '../data/MovementAAL/dataset/MovementAAL_target.csv'
group_path = '../data/MovementAAL/groups/MovementAAL_DatasetGroup.csv'

### -------------------------------------------------------- loading files
file_dfs = []
file_lens = []

for i in range(1,315):
    file_path = input_path + str(i) + '.csv'
    file_df = pd.read_csv(file_path, header=0).reset_index(drop=True)
    file_df['_id'] = i 
    file_df = file_df.reset_index(drop=True)
    
   
    file_dfs.append(file_df)
    file_lens.append(file_df.shape[0])
    

input_df = pd.concat(file_dfs)
target_df = pd.read_csv(target_path)[' class_label']
group_df = pd.read_csv(group_path)[' dataset_ID']

gr_li = []
for idx, (id_num, target, group) in enumerate(zip(input_df['_id'].unique(), target_df, group_df)):
    gr = input_df[input_df['_id'] == id_num]
    gr = gr.reset_index(drop=True)
    gr['target'] = [target]*gr.shape[0]
    gr['group'] = [group]*gr.shape[0]
    gr['time'] = np.arange(0, gr.shape[0]/8, 1/8)
    gr_li.append(gr)


df = pd.concat(gr_li)
df = df.reset_index(drop=True)

# adding target label and group label
df['target_label'] = df['target'].apply(lambda x: 'Movement' if x==1 else 'Non-Movement')
df['group_label'] = (
                    np.select(condlist = [df['group']==1, df['group']==2, df['group']==3],
                             choicelist=['environment_1', 'environment_2', 'environment_3'])
                    )
# rearranging the columns
df = df.rename(columns={'#RSS_anchor1':'RSS_anchor1', ' RSS_anchor2':'RSS_anchor2', ' RSS_anchor3':'RSS_anchor3', ' RSS_anchor4':'RSS_anchor4'}) 

df = df[['_id', 'time', 'RSS_anchor1', 'RSS_anchor2', 'RSS_anchor3', 'RSS_anchor4',
        'target', 'target_label', 'group', 'group_label']]

# saving the preprocessed file
df.to_csv('../data/indoor_movement.csv', index = False)

## Visualization

In [91]:
df_red = df[df['_id'].isin(np.arange(25, 30))]
df_red.to_csv('../data/indoor_movement_red.csv', index = False)

# signal visulalization

i=1
fig = px.line(df_red, x="time", y="RSS_anchor{}".format(i), color="target_label", line_group="_id", 
              hover_name="target", line_shape="linear", width = 950, height = 550, line_dash = '_id',
              facet_row = 'target_label', color_discrete_sequence = px.colors.qualitative.Alphabet)
iplot(fig)


In [ ]:
# histogram and density plot


In [92]:
# dimensionality decomposition
input_cols = df[['RSS_anchor1', 'RSS_anchor2', 'RSS_anchor3', 'RSS_anchor4']]
target_col = df[['target_label']]
target_group = df[['group_label']]

pca = PCA(n_components=2, random_state=42)
tsne = TSNE(n_components=2, random_state=42)
umap = UMAP(n_components=2, random_state=42)

pca_res = pca.fit_transform(input_cols)
pca_res = pd.DataFrame(pca_res, columns = ['pc1', 'pc2'])
pca_res = pd.concat([pca_res, target_col,target_group], axis=1, sort = False)
pca_res = pca_res.sample(n=1000, random_state=42)

tsne_res = tsne.fit_transform(input_cols)
tsne_res = pd.DataFrame(tsne_res, columns = ['tsne_x', 'tsne_y'])
tsne_res = pd.concat([tsne_res, target_col,target_group], axis=1, sort = False)
tsne_res = tsne_res.sample(n=1000, random_state=42)

umap_res = umap.fit_transform(input_cols)
umap_res = pd.DataFrame(umap_res, columns = ['umap_x', 'umap_y'])
umap_res = pd.concat([umap_res, target_col,target_group], axis=1, sort = False)
umap_res = umap_res.sample(n=1000, random_state=42)

In [97]:
px.scatter(pca_res, x="pc1", y="pc2", color="target_label", 
              hover_name="target_label", width = 990, height = 500,
           color_discrete_sequence = px.colors.qualitative.Set1, facet_col = 'group_label'
              )


In [98]:
px.scatter(tsne_res, x="tsne_x", y="tsne_y", color="target_label", 
              hover_name="target_label", width = 990, height = 500,
           color_discrete_sequence = px.colors.qualitative.Set1, facet_col = 'group_label'
              )

In [99]:
px.scatter(umap_res, x="umap_x", y="umap_y", color="target_label", 
              hover_name="target_label", width = 990, height = 550,
           color_discrete_sequence = px.colors.qualitative.Set1, facet_col = 'group_label'
              )

In [96]:
### Classification

In [100]:
help(px.scatter)

Help on function scatter in module plotly_express._chart_types:

scatter(data_frame, x=None, y=None, color=None, opacity=None, symbol=None, size=None, hover_name=None, text=None, facet_row=None, facet_col=None, error_x=None, error_x_minus=None, error_y=None, error_y_minus=None, animation_frame=None, animation_group=None, category_orders={}, labels={}, color_discrete_sequence=['#636efa', '#EF553B', '#00cc96', '#ab63fa', '#19d3f3', '#e763fa', '#fecb52', '#ffa15a', '#ff6692', '#b6e880'], color_discrete_map={}, color_continuous_scale=['#0508b8', '#1910d8', '#3c19f0', '#6b1cfb', '#981cfd', '#bf1cfd', '#dd2bfd', '#f246fe', '#fc67fd', '#fe88fc', '#fea5fd', '#febefe', '#fec3fe'], color_continuous_midpoint=None, symbol_sequence=['circle', 'diamond', 'square', 'x', 'cross'], symbol_map={}, size_max=20, marginal_x=None, marginal_y=None, trendline=None, trendline_color_override=None, log_x=False, log_y=False, range_x=None, range_y=None, render_mode='auto', title=None, template='plotly', width=None